In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
import os
from configs.config_spottune import CFG
from spottunet.split import one2one
from pathlib import Path
from collections import defaultdict
import torch

### Policy Visualization

In [ ]:
def floats2colors(floats, bins=256):
    palette = sns.color_palette(palette='magma', n_colors=bins).as_hex()
    ints = np.int64(floats * (bins - 1))
    colors = [palette[i] for i in ints]
    return colors

def get_stats_spottune(exp_path, fold='inference'):
    p = torch.load(Path(exp_path) / f'policy_{fold}_record/policy_record')
    f = open(Path(exp_path) / f'policy_{fold}_record/iter_record', 'r')
    n_iter = f.read()
    f.close()
    record = (p / int(n_iter)).detach().numpy()
    return record


ns = np.arange(15,20)
path_template = 'spottune_results/baseline_results/{}/'
paths = [#"baseline_focal_lovasz_adam_default_Posterize_exp_ssi_1_nid_3_default",
         #"baseline_focal_lovasz_adam_default_Posterize_exp_ssi_12_nid_2_default",
         "baseline_focal_lovasz_adam_default_exp_ssi_48_nid_1_default",
         "baseline_focal_lovasz_adam_default_exp_ssi_12_nid_2_default",
         "baseline_focal_lovasz_adam_default_exp_ssi_1_nid_3_default",
        "baseline_focal_lovasz_adam_rand_aug_default_v1_exp_ssi_48_nid_1_default",
         "baseline_focal_lovasz_adam_rand_aug_default_v1_exp_ssi_12_nid_2_default",
        "baseline_focal_lovasz_adam_rand_aug_default_v1_exp_ssi_1_nid_3_default"
        ]

records = defaultdict(list)
for k,v in enumerate(paths):
    for n in ns:
        try:
            records[v].append(get_stats_spottune(Path(path_template.format(v)) / f'mode_{n}'))
        except:
            None
        
records_mean = {k: 1 - np.mean(v, axis=0) for k, v in records.items()}

In [ ]:
records_mean

In [ ]:
plt.figure(figsize=(20,10))
sns.barplot(data=np.array([np.array(records["exp_ssi_48_nid_1_gamma"])[0],np.array(records["exp_ssi_48_nid_1_gamma"])[0]]) )

In [ ]:
colors = {k: floats2colors(v) for k, v in records_mean.items()}
print(-1, *[f'{k:10s}' for k in colors.keys()])
for i in range(1, len(colors[paths[0]]) + 1):
    print(f'{i:2d}', *[f'{v[i - 1]:10s}' for _, v in colors.items()])

In [ ]:
sns.palplot(colors['baseline_focal_lovasz_adam_default_exp_ssi_48_nid_1_default'])
sns.palplot(colors['baseline_focal_lovasz_adam_default_exp_ssi_12_nid_2_default'])
sns.palplot(colors['baseline_focal_lovasz_adam_default_exp_ssi_1_nid_3_default'])

sns.palplot(colors['baseline_focal_lovasz_adam_rand_aug_default_v1_exp_ssi_48_nid_1_default'])
sns.palplot(colors['baseline_focal_lovasz_adam_rand_aug_default_v1_exp_ssi_12_nid_2_default'])
sns.palplot(colors['baseline_focal_lovasz_adam_rand_aug_default_v1_exp_ssi_1_nid_3_default'])


In [ ]:
sns.palplot(colors['baseline_focal_lovasz_SGD_default_None_exp_ssi_1_nid_3_default'])

In [ ]:
colors = {k: floats2colors(v) for k, v in records_mean.items()}
print(-1, *[f'{k:10s}' for k in colors.keys()])
for i in range(1, len(colors[paths[0]]) + 1):
    print(*[f'{v[i - 1]:10s}' for _, v in colors.items()])

In [ ]:
2*256//12

### Dice Predictions

In [ ]:

with open("spottune_results/exp_ssi_48_nid_1_norm/mode_3/sdice_score.json","r") as f:
    data = json.load(f)
list(data.values())

### Spottune Performance 

In [ ]:
meta = pd.read_csv('meta.csv', index_col='id')
df_id = pd.read_csv('meta.csv')
seed = 0xBadCafe
pretrained = True
n_first_exclude = 5
n_exps = 30
split = one2one(df_id, val_size=CFG.val_size, n_add_ids=CFG.n_add_ids,
            train_on_add_only=pretrained, seed=seed)[n_first_exclude:n_exps]

In [ ]:
### Siemens3
mode_domain = ["Siemens15", "Ge15", "Ge3", "philips15", "philips3"]
paths = ["exp_03_no_aug", "exp_03_rrc", "exp_03_gaussianblur", "exp_03_ssi_48_nid_1_gamma"]

results = []
for path in paths:
    mode_dict = {}
    modes = [0,1,2,3,4]
    for mode in modes:
        
        with open(f"spottune_results/{path}/mode_{str(mode)}/sdice_score.json","r") as f:
            data = json.load(f)
            mode_dict[mode_domain[mode]] = np.mean(list(data.values()))
    mode_dict[' '] = path
    results.append(mode_dict)
results.append({"Siemens15": 0.849, "Ge15": 0.937, "Ge3": 0.422, "philips15": 0.743, "philips3": 0.644, " ": "baseline"})

In [ ]:
### Ge3

def get_baseline_target_domains(name, path, fold, meta):
    with open(f"baseline_results/{path}/mode_{str(fold)}/sdice_score.json","r") as f:
        sdices = json.load(f)
    mode_dict = {}
    for t in sorted(set(meta['fold'].unique()) - {fold}):
        df_row = meta[meta['fold'] == t].iloc[0]
        target_name = df_row['tomograph_model'] + str(df_row['tesla_value'])
        ids_t = meta[meta['fold'] == t].index
        mode_dict[target_name] = np.round(np.mean([sdsc for _id, sdsc in sdices.items() if _id in ids_t]),3)
    mode_dict[' '] = name
    return mode_dict

def get_target_domains(name, path, folds, meta, n_first_exclude=5, n_folds=6):
    mode_dict = {}
    for fold in sorted(folds):
        subset_df = df_id.iloc[split[fold][2]]
        target_name = str(subset_df['tomograph_model'].unique()[0]) + str(subset_df['tesla_value'].unique()[0])
        ids_t = subset_df['id'].unique()
        try:
            with open(f"{path}/mode_{str(fold)}/sdice_score.json","r") as f:
                sdices = json.load(f)
            mode_dict[target_name] = np.round(np.mean([sdsc for _id, sdsc in sdices.items() if _id in ids_t]),3)
        except:
            mode_dict[target_name] = np.nan
    mode_dict[' '] = name
    return mode_dict

### Old experiments

In [ ]:
baselines = ["baseline23_exp4_default", "baseline23_exp4_rrc", "baseline23_exp4_weak_rrc",
             "baseline23_jacard_default", "baseline_lovasz_default",
             "baseline23_exp3_gamma", "baseline23_exp3_gaus", 
             "baseline_gamma", "baseline_gaus", 
             "baseline_rotation", 
             "baseline_translation",
             "baseline_cutout", "baseline_brightness_contrast",
             "baseline23_exp3_noSmoothing_mixup"]
mode_domain = ["Siemens15", "Ge15", "Ge3", "philips15", "philips3"]
paths = [
          "exp_ssi_48_nid_1_default","exp_ssi_12_nid_2_default","exp_ssi_1_nid_3_default",
          "exp_ssi_48_nid_1_gamma", "exp_ssi_12_nid_2_gamma","exp_ssi_1_nid_3_gamma",
         "exp_ssi_48_nid_1_gaus",  "exp_ssi_12_nid_2_gaus", "exp_ssi_1_nid_3_gaus",
         "exp_ssi_48_nid_1_brightness_contrast", "exp_ssi_12_nid_2_brightness_contrast","exp_ssi_1_nid_3_brightness_contrast",
          "exp_ssi_48_nid_1_translate", "exp_ssi_12_nid_2_translate", "exp_ssi_1_nid_3_translate",
          "exp_ssi_48_nid_1_rotation", "exp_ssi_12_nid_2_rotation", "exp_ssi_1_nid_3_rotation",
         "exp_ssi_48_nid_1_cutout", "exp_ssi_12_nid_2_cutout", "exp_ssi_1_nid_3_cutout",
        ]
results = []
s = 3
for baseline_path in baselines:
    baseline_scores = get_baseline_target_domains(baseline_path,baseline_path,s,meta)
    results.append(baseline_scores)
for p in paths:
    modes = [10,11,12,13,14]

    path = os.path.join("spottune_results",p)
    spottune_scores = get_target_domains(p,path,modes,meta)
    
    results.append(spottune_scores)
    
df = pd.DataFrame.from_records(results, index=' ')
df['Average'] = np.mean(df, axis=1)
df['avg. gain'] = np.mean(df.iloc[1:] - df.iloc[0], axis=1)
df

### new experiemnts spottune

In [ ]:
mode_domain = ["Siemens15", "Ge15", "Ge3", "philips15", "philips3"]
baselines = ["baseline_focal_lovasz_adam_default"]
paths = ["baseline_focal_lovasz_adam_default_exp_ssi_48_nid_1_default",
         "baseline_focal_lovasz_adam_default_exp_ssi_12_nid_2_default",
         "baseline_focal_lovasz_adam_default_exp_ssi_1_nid_3_default",
        ]
results = []
s = 4
for baseline_path in baselines:
    baseline_scores = get_baseline_target_domains(baseline_path,baseline_path,s,meta)
    results.append(baseline_scores)
for p in paths:
    modes = [15,16,17,18,19]

    path = os.path.join("spottune_results/baseline_results/",p)
    spottune_scores = get_target_domains(p,path,modes,meta)
    
    results.append(spottune_scores)
    
df = pd.DataFrame.from_records(results, index=' ')
df['Average'] = np.mean(df, axis=1)
df['avg. gain'] = np.mean(df.iloc[1:] - df.iloc[0], axis=1)
df

In [ ]:
mode_domain = ["Siemens15", "Ge15", "Ge3", "philips15", "philips3"]
baselines = ["baseline_focal_lovasz_adam_rand_aug_default_v1"]
paths = ["baseline_focal_lovasz_adam_rand_aug_default_v1_exp_ssi_48_nid_1_default",
         "baseline_focal_lovasz_adam_rand_aug_default_v1_exp_ssi_12_nid_2_default",
         "baseline_focal_lovasz_adam_rand_aug_default_v1_exp_ssi_1_nid_3_default",
        ]
results = []
s = 4
for baseline_path in baselines:
    baseline_scores = get_baseline_target_domains(baseline_path,baseline_path,s,meta)
    results.append(baseline_scores)
for p in paths:
    modes = [15,16,17,18,19]

    path = os.path.join("spottune_results/baseline_results/",p)
    spottune_scores = get_target_domains(p,path,modes,meta)
    
    results.append(spottune_scores)
    
df = pd.DataFrame.from_records(results, index=' ')
df['Average'] = np.mean(df, axis=1)
df['avg. gain'] = np.mean(df.iloc[1:] - df.iloc[0], axis=1)
df

In [ ]:
baselines = ["baseline_lovasz_nostopping_default",
             "baseline_focal_lovasz_nofp_default", "baseline_focal_lovasz_adam_default"]
mode_domain = ["Siemens15", "Ge15", "Ge3", "philips15", "philips3"]
results = []
s = 3
for baseline_path in baselines:
    baseline_scores = get_baseline_target_domains(baseline_path,baseline_path,s,meta)
    results.append(baseline_scores)
df = pd.DataFrame.from_records(results, index=' ')
df['Average'] = np.mean(df, axis=1)
df['avg. gain'] = np.mean(df.iloc[1:] - df.iloc[0], axis=1)
df['avg_std'] = np.std(df.iloc[1:] - df.iloc[0], axis=1)
df

In [ ]:
baselines = ["baseline_focal_lovasz_adam_default",
             "baseline_focal_lovasz_adam_rand_aug_default_v1",
             "baseline_focal_lovasz_Adam_rand_aug_default",
             "baseline_focal_lovasz_adam_rand_aug_combo1",
             "baseline_focal_lovasz_adam_default_Brightness",
             "baseline_focal_lovasz_adam_default_Contrast",
             "baseline_focal_lovasz_adam_default_SolarizeSolarizeAdd",
            "baseline_focal_lovasz_adam_default_Posterize",
            "baseline_focal_lovasz_adam_default_Rotate",
            "baseline_focal_lovasz_adam_default_TranslateXRelTranslateYRel",
            "baseline_focal_lovasz_adam_default_ShearXShearY",
            "baseline_focal_lovasz_adam_default_Sharpness",
            "baseline_focal_lovasz_adam_totvar",
            "baseline_focal_lovasz_adam_GridDistortion",
            "baseline_focal_lovasz_adam_gamma",
            #"baseline_focal_lovasz_Adam_gamma_fcm_all",
            #"baseline_focal_lovasz_Adam_gamma_fcm_wm",
            #"baseline_focal_lovasz_Adam_gamma_fcm_csf",
            #"baseline_focal_lovasz_Adam_gamma_fcm_gm",
            "baseline_focal_lovasz_adam_cutout",
            "baseline_focal_lovasz_adam_default_mixup0.2",
            ]
mode_domain = ["Siemens15", "Ge15", "Ge3", "philips15", "philips3"]
results = []
s = 4
for baseline_path in baselines:
    baseline_scores = get_baseline_target_domains(baseline_path,baseline_path,s,meta)
    results.append(baseline_scores)
df = pd.DataFrame.from_records(results, index=' ')
df['Average'] = np.mean(df, axis=1)
df['avg. gain'] = np.mean(df.iloc[1:] - df.iloc[0], axis=1)
df['avg_std'] = np.std(df.iloc[1:] - df.iloc[0], axis=1)
df

In [ ]:
df.loc[:,'avg. gain'].values[2:].sum()

In [ ]:
baselines = ["baseline_focal_lovasz_adam_default",
             #"baseline_focal_lovasz_SGD_default",
             "baseline_focal_lovasz_adam_rand_aug_default_v1",
             "baseline_focal_lovasz_adam_default_Brightness",
             "baseline_focal_lovasz_adam_default_Contrast",
             "baseline_focal_lovasz_adam_default_SolarizeSolarizeAdd",
            "baseline_focal_lovasz_adam_default_Posterize",
            "baseline_focal_lovasz_adam_default_Rotate",
            "baseline_focal_lovasz_adam_default_TranslateXRelTranslateYRel",
            "baseline_focal_lovasz_adam_default_ShearXShearY",
            "baseline_focal_lovasz_adam_default_Sharpness"]
mode_domain = ["Siemens15", "Ge15", "Ge3", "philips15", "philips3"]
results = []
s = 4
for baseline_path in baselines:
    baseline_scores = get_baseline_target_domains(baseline_path,baseline_path,s,meta)
    results.append(baseline_scores)
df = pd.DataFrame.from_records(results, index=' ')
df['Average'] = np.mean(df, axis=1)
df['avg. gain'] = np.mean(df.iloc[1:] - df.iloc[0], axis=1)
df['avg_std'] = np.std(df.iloc[1:] - df.iloc[0], axis=1)
df['Source Domain'] = ['Philips1.5T']*len(df)
df

In [ ]:
results = []
s = 0
for baseline_path in baselines:
    baseline_scores = get_baseline_target_domains(baseline_path,baseline_path,s,meta)
    results.append(baseline_scores)
df2 = pd.DataFrame.from_records(results, index=' ')
df2['Average'] = np.mean(df2, axis=1)
df2['avg. gain'] = np.mean(df2.iloc[1:] - df2.iloc[0], axis=1)
df2['avg_std'] = np.std(df2.iloc[1:] - df2.iloc[0], axis=1)
df2['Source Domain'] = ['Siemens1.5T']*len(df2)
df2

In [ ]:
both = pd.concat((df, df2))
df_rand_aug = both.loc["baseline_focal_lovasz_adam_rand_aug_default_v1"]
df_rand_aug.reset_index(level=0, inplace=True)
df_single_augs = both.drop(index=["baseline_focal_lovasz_adam_default","baseline_focal_lovasz_adam_rand_aug_default_v1"])
df_single_augs.reset_index(level=0, inplace=True)
df_single_augs

In [ ]:
sns.set_style("darkgrid")

fig, axes = plt.subplots(1,2,gridspec_kw={'width_ratios': [1, 3]})
plt.tight_layout()
sns.barplot(ax=axes[0],data=df_rand_aug,x=' ',y='avg. gain',hue='Source Domain')
axes[0].set_xticklabels(['RandAug'], rotation=45,ha='right', rotation_mode='anchor')
axes[0].set_yticks([0,0.025,0.05,0.075,0.10,0.125,0.15])
axes[0].set_ylim(0,0.15)
axes[0].get_legend().remove()
axes[0].set_ylabel("Avg. increase (SDice)")


sns.barplot(ax=axes[1],data=df_single_augs,x=' ',y='avg. gain',hue='Source Domain')
axes[1].set_xticklabels(['Brightness', 'Contrast', 'Solarize', 'Posterize','Rotate','Translate','Shear', 'Sharpness'], rotation=45,ha='right', rotation_mode='anchor')
axes[1].set_yticks([0,0.025,0.05,0.075,0.10,0.125,0.15])
axes[1].set_ylim(0,0.15)
axes[1].set_ylabel("")




plt.savefig('filename.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df_rand_aug = both.loc["baseline_focal_lovasz_adam_rand_aug_default_v1"]
df_rand_aug.reset_index(level=0, inplace=True)
df_single_augs = both.drop(index=["baseline_focal_lovasz_adam_default","baseline_focal_lovasz_adam_rand_aug_default_v1"])
df_single_augs.reset_index(level=0, inplace=True)

fig, axes = plt.subplots(1,2,gridspec_kw={'width_ratios': [3, 1]})
fig.tight_layout()
sns.boxplot(ax=axes[0],data=df_single_augs,x=' ',y='avg. gain',hue='fold',)
axes[0].set_xticklabels(np.unique(df_single_augs[' ']), rotation=45,ha='right', rotation_mode='anchor')
sns.boxplot(ax=axes[1],data=df_rand_aug,x=' ',y='avg. gain',hue='fold')
axes[1].set_xticklabels(np.unique(df_rand_aug[' ']), rotation=45,ha='right', rotation_mode='anchor')
plt.show()

### Original SGD

In [ ]:

def get_SGD_baseline_target_domains(name, path, fold, meta):
    with open(f"{path}/experiment_{str(fold)}/test_metrics/sdice_score.json","r") as f:
        sdices = json.load(f)
    mode_dict = {}
    for t in sorted(set(meta['fold'].unique()) - {fold}):
        df_row = meta[meta['fold'] == t].iloc[0]
        target_name = df_row['tomograph_model'] + str(df_row['tesla_value'])
        ids_t = meta[meta['fold'] == t].index
        mode_dict[target_name] = np.round(np.mean([sdsc for _id, sdsc in sdices.items() if _id in ids_t]),3)
    mode_dict[' '] = name
    return mode_dict

def get_SGD_target_domains(name, path, folds, meta, n_first_exclude=5, n_folds=6):
    mode_dict = {}
    for fold in sorted(folds):
        subset_df = df_id.iloc[split[fold][2]]
        target_name = str(subset_df['tomograph_model'].unique()[0]) + str(subset_df['tesla_value'].unique()[0])
        ids_t = subset_df['id'].unique()

        with open(f"{path}/experiment_{str(fold)}/test_metrics/sdice_score.json","r") as f:
            sdices = json.load(f)
        mode_dict[target_name] = np.round(np.mean([sdsc for _id, sdsc in sdices.items() if _id in ids_t]),3)

        #mode_dict[target_name] = np.nan
    mode_dict[' '] = name
    return mode_dict

mode_domain = ["Siemens15", "Ge15", "Ge3", "philips15", "philips3"]
baselines = ["../../domain_shift_anatomy/dart_results/baseline_default",
            ]
paths = ["spottune_rand_aug/test_k_reg_opt_1_48",
         "spottune_rand_aug/test_k_reg_opt_2_12",
         "spottune_rand_aug/test_k_reg_opt_3_1",
        ]
results = []
s = 1
for baseline_path in baselines:
    baseline_scores = get_SGD_baseline_target_domains(baseline_path,baseline_path,s,meta)
    results.append(baseline_scores)
for p in paths:
    modes = [0,1,2,3,4]

    path = os.path.join("../../domain_shift_anatomy/dart_results",p)
    spottune_scores = get_SGD_target_domains(p,path,modes,meta)
    
    results.append(spottune_scores)
    
df = pd.DataFrame.from_records(results, index=' ')
df['Average'] = np.mean(df, axis=1)
df['avg. gain'] = np.mean(df.iloc[1:] - df.iloc[0], axis=1)
df

In [ ]:
mode_domain = ["Siemens15", "Ge15", "Ge3", "philips15", "philips3"]
baselines = ["../../domain_shift_anatomy/dart_results/baseline"
            ]
paths = ["spottune_rand_aug_v2/test_k_reg_opt_1_48",
         "spottune_rand_aug_v2/test_k_reg_opt_2_12",
         "spottune_rand_aug_v2/test_k_reg_opt_3_1",
        ]
results = []
s = 1
for baseline_path in baselines:
    baseline_scores = get_SGD_baseline_target_domains(baseline_path,baseline_path,s,meta)
    results.append(baseline_scores)
for p in paths:
    modes = [0,1,2,3,4]

    path = os.path.join("../../domain_shift_anatomy/dart_results",p)
    spottune_scores = get_SGD_target_domains(p,path,modes,meta)
    
    results.append(spottune_scores)
    
df = pd.DataFrame.from_records(results, index=' ')
df['Average'] = np.mean(df, axis=1)
df['avg. gain'] = np.mean(df.iloc[1:] - df.iloc[0], axis=1)
df